In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, KFold

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [2]:
train_df = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Train.csv')
test_df = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Test.csv')

train_df.set_index('inst_id', inplace=True)
test_df.set_index('inst_id', inplace=True)

train_df.head()

,OC,sido_choongchung,sido_gangwon,sido_gyeongin,sido_gyeongsang,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,...,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange,bedCount_class
inst_id,,,,,,,,,,,,,,,,,,,,,
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.222130,0.000000,20.099863,19.781657,21.686169,20.963246,4.143135,4.174387,0.0,3.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,18.841626,9.180719,18.812726,17.568559,18.838180,12.258481,6.687109,6.701960,0.0,4.0
4,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.459586,0.693147,0.0,4.0
7,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,23.557572,22.944579,23.755157,23.437961,23.284691,22.769775,6.498282,6.498282,0.0,4.0
9,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,24.126880,23.590810,23.784786,23.370010,15.531458,22.923325,5.332719,5.288267,0.0,3.0


In [3]:
test_df.head()

,OC,sido_choongchung,sido_gyeongin,sido_gyeongsang,sido_jeju,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,...,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange,bedCount_class
inst_id,,,,,,,,,,,,,,,,,,,,,
2,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,24.648885,24.338050,22.275075,22.268698,23.501178,22.914314,6.542472,6.542472,0.0,3.0
5,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,24.463273,23.909618,23.950510,23.630167,22.055404,0.000000,5.940171,5.918894,0.0,2.0
6,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,24.047434,23.806220,0.000000,0.000000,0.000000,23.519657,5.285888,5.225619,0.2,3.0
8,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,22.980460,22.170624,21.662855,0.000000,24.390483,0.000000,6.634633,6.634633,0.0,4.0
10,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,22.742900,21.916712,24.318498,23.784183,23.971502,23.436150,6.082219,5.955837,0.0,3.0


In [4]:
train_lr = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Train_lr.csv')
train_lr.set_index('inst_id', inplace=True)

train_lr.head()

,sido_choongchung,sido_gyeongsang,sido_jeonra,instkind_nursing_hospital,instkind_traditional_hospital,revenue1,sga1,salary1,noe1,interest1,...,profit2,inventoryAsset2,OnonCAsset2,shortLoan2,NCLiabilities2,longLoan2,surplus2,ownerChange,bedCount_class,OC
inst_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,0.0,22.162515,22.099796,21.433189,16.539187,16.398469,...,19.152229,16.425505,18.798422,0.000000,20.099863,19.781657,20.963246,0.0,3.0,1
3,0.0,1.0,0.0,0.0,0.0,23.980280,23.481393,19.521179,16.823799,15.443340,...,17.025473,16.044684,15.372413,9.180719,18.812726,17.568559,12.258481,0.0,4.0,1
4,0.0,0.0,0.0,1.0,0.0,20.727778,19.918561,19.507228,10.308986,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,4.0,1
7,0.0,0.0,0.0,0.0,0.0,25.006954,24.981392,24.182293,20.954034,20.902985,...,20.546991,20.934483,22.088313,22.944579,23.755157,23.437961,22.769775,0.0,4.0,1
9,0.0,1.0,0.0,0.0,0.0,24.615974,24.587275,23.920337,21.073281,20.991191,...,17.301128,19.908536,20.486709,23.590810,23.784786,23.370010,22.923325,0.0,3.0,1


In [5]:
train_dt = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Train_dt.csv')
train_dt.set_index('inst_id', inplace=True)

train_dt.head()

,instkind_hospital,instkind_traditional_clinic,instkind_nan,revenue1,salescost1,sga1,salary1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset2,receivableL2,employee2,OC
inst_id,,,,,,,,,,,,,,
1,0.0,0.0,0.0,22.162515,0.000000,22.099796,21.433189,20.735886,20.720935,19.968362,16.425505,0.0,4.174387,1
3,0.0,0.0,0.0,23.980280,9.659782,23.481393,19.521179,19.074269,19.037270,10.917985,16.044684,0.0,6.701960,1
4,0.0,0.0,0.0,20.727778,20.060616,19.918561,19.507228,19.422937,19.351593,15.906875,0.000000,0.0,0.693147,1
7,0.0,0.0,0.0,25.006954,0.000000,24.981392,24.182293,23.291406,23.168630,0.000000,20.934483,0.0,6.498282,1
9,0.0,0.0,0.0,24.615974,0.000000,24.587275,23.920337,22.566524,22.493677,22.134087,19.908536,0.0,5.288267,1


In [6]:
train_dt.shape

(301, 14)

# 1) 67개 columns

In [51]:
X_train = train_df.drop('OC', axis=1)
y_train = train_df['OC']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [52]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_tr_over, y_tr_over = smote.fit_resample(X_train, y_train)

#### 1) 교차 검증

In [53]:
kfold = KFold(n_splits=5, shuffle = True, random_state=0)

base_model = DecisionTreeClassifier(max_depth = 1)    # stump tree
clf11 = AdaBoostClassifier(base_estimator = base_model, random_state=42)

scores = cross_val_score(clf11 , X_tr_over, y_tr_over, cv=kfold)

# Training data에 대한 성능
print('Cross-validation Accuracy:',np.round(scores, 4))
print('Average of Validation Accuracy:', np.round(np.mean(scores), 4))

Cross-validation Accuracy: [0.9783 0.9891 0.9674 0.978  0.967 ]
Average of Validation Accuracy: 0.976


#### 2) accuracy 외의 지표

In [54]:
clf12 = AdaBoostClassifier(base_estimator = base_model, random_state=42)
clf12.fit(X_tr_over, y_tr_over)
pred = clf12.predict(X_val)

print('train accuracy : {:.4f}'.format(clf12.score(X_tr_over, y_tr_over)))
print('val accuracy : {:.4f}'.format(accuracy_score(y_val, pred)))

train accuracy : 1.0000
val accuracy : 0.9344


In [13]:
def clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print()
    print('정확도 : {:.4f}\n정밀도 : {:.4f}\n재현율 : {:.4f}'.format(accuracy, precision, recall))

In [55]:
clf_eval(y_val, pred)

오차행렬
[[ 1  3]
 [ 1 56]]

정확도 : 0.9344
정밀도 : 0.9492
재현율 : 0.9825


# 2) Train_lr

In [56]:
X_train = train_lr.drop('OC', axis=1)
y_train = train_lr['OC']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [57]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_tr_over, y_tr_over = smote.fit_resample(X_train, y_train)

#### 1) 교차 검증

In [58]:
kfold = KFold(n_splits=5, shuffle = True, random_state=0)
clf21 = AdaBoostClassifier(base_estimator = base_model, random_state=42)

scores = cross_val_score(clf21 , X_tr_over, y_tr_over, cv=kfold)

# Training data에 대한 성능
print('Cross-validation Accuracy:',np.round(scores, 4))
print('Average of Validation Accuracy:', np.round(np.mean(scores), 4))

Cross-validation Accuracy: [0.9783 0.9457 0.9783 0.989  0.967 ]
Average of Validation Accuracy: 0.9716


#### 2) accuracy 외의 지표

In [59]:
clf22 = AdaBoostClassifier(base_estimator = base_model, random_state=42)
clf22.fit(X_tr_over, y_tr_over)
pred = clf22.predict(X_val)

print('train accuracy : {:.4f}'.format(clf22.score(X_train, y_train)))
print('val accuracy : {:.4f}'.format(accuracy_score(y_val, pred)))

train accuracy : 1.0000
val accuracy : 0.9344


In [60]:
clf_eval(y_val, pred)

오차행렬
[[ 1  3]
 [ 1 56]]

정확도 : 0.9344
정밀도 : 0.9492
재현율 : 0.9825


# 3) Train_dt

In [61]:
X_train = train_dt.drop('OC', axis=1)
y_train = train_dt['OC']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [62]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_tr_over, y_tr_over = smote.fit_resample(X_train, y_train)

#### 1) 교차 검증

In [63]:
kfold = KFold(n_splits=5, shuffle = True, random_state=0)
clf31 = AdaBoostClassifier(base_estimator = base_model, random_state=42)

scores = cross_val_score(clf31 , X_tr_over, y_tr_over, cv=kfold)

# Training data에 대한 성능
print('Cross-validation Accuracy:',np.round(scores, 4))
print('Average of Validation Accuracy:', np.round(np.mean(scores), 4))

Cross-validation Accuracy: [0.9022 0.9565 0.9565 0.9121 0.9231]
Average of Validation Accuracy: 0.9301


#### 2) accuracy 외의 지표

In [64]:
clf32 = AdaBoostClassifier(base_estimator = base_model, random_state=42)
clf32.fit(X_tr_over, y_tr_over)
pred = clf32.predict(X_val)

print('train accuracy : {:.4f}'.format(clf32.score(X_tr_over, y_tr_over)))
print('val accuracy : {:.4f}'.format(accuracy_score(y_val, pred)))

train accuracy : 0.9869
val accuracy : 0.8689


In [65]:
clf_eval(y_val, pred)

오차행렬
[[ 0  4]
 [ 4 53]]

정확도 : 0.8689
정밀도 : 0.9298
재현율 : 0.9298
